In [1]:
import pandas as pd
import os
import numpy as np

#model
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler,PowerTransformer
from sklearn.feature_selection import f_classif,mutual_info_classif,VarianceThreshold,SequentialFeatureSelector, SelectPercentile
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score,recall_score, roc_auc_score

import optuna
from imblearn.over_sampling import SMOTE,ADASYN
import matplotlib.pyplot as plt

import pickle
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline

/home/riccardo/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Dataset with clinical Data

clinical_db_center1 = pd.read_excel('clinical_center1.xlsx',index_col=0)
clinical_db_center2 = pd.read_excel('clinical_center2.xlsx',index_col=0)

In [3]:
# Center 1

HBP_core_center1 = pd.read_excel('center1_HBP_core.xlsx',index_col=0)
selected_cols = HBP_core_center1.columns


HBP_ring_center1 = pd.read_excel('center1_HBP_ring.xlsx',index_col=0)[selected_cols]
PVP_core_center1 = pd.read_excel('center1_PVP_core.xlsx',index_col=0)[selected_cols]
PVP_ring_center1 = pd.read_excel('center1_PVP_ring.xlsx',index_col=0)[selected_cols]


HBP_core_center1['TRG'] = [1 if x<=3 else 0 for x in clinical_db_center1['TRG']]
HBP_ring_center1['TRG'] = [1 if x<=3 else 0 for x in clinical_db_center1['TRG']]
PVP_core_center1['TRG'] = [1 if x<=3 else 0 for x in clinical_db_center1['TRG']]
PVP_ring_center1['TRG'] = [1 if x<=3 else 0 for x in clinical_db_center1['TRG']]



# Center 2

HBP_core_center2 = pd.read_excel('center2_HBP_core.xlsx',index_col=0)
selected_cols = HBP_core_center2.columns


HBP_ring_center2 = pd.read_excel('center2_HBP_ring.xlsx',index_col=0)[selected_cols]
PVP_core_center2 = pd.read_excel('center2_PVP_core.xlsx',index_col=0)[selected_cols]
PVP_ring_center2 = pd.read_excel('center2_PVP_ring.xlsx',index_col=0)[selected_cols]


HBP_core_center2['TRG'] = [1 if x<=3 else 0 for x in clinical_db_center2['TRG']]
HBP_ring_center2['TRG'] = [1 if x<=3 else 0 for x in clinical_db_center2['TRG']]
PVP_core_center2['TRG'] = [1 if x<=3 else 0 for x in clinical_db_center2['TRG']]
PVP_ring_center2['TRG'] = [1 if x<=3 else 0 for x in clinical_db_center2['TRG']]


In [4]:
# ICH



HBP_core = pd.concat([HBP_core_center1,HBP_core_center2])
HBP_ring = pd.concat([HBP_ring_center1,HBP_ring_center2])
PVP_core = pd.concat([PVP_core_center1,PVP_core_center2])
PVP_ring = pd.concat([PVP_ring_center1,PVP_ring_center2])

HBP = HBP_core.join(HBP_ring.drop(['TRG'],axis = 1), rsuffix='_ring')
PVP = PVP_core.join(PVP_ring.drop(['TRG'],axis = 1), rsuffix='_ring')

ring = HBP_ring.join(PVP_ring.drop(['TRG'],axis = 1), rsuffix='_pvp')
core = HBP_core.join(PVP_core.drop(['TRG'],axis = 1), rsuffix='_pvp')

total = HBP_core.join(HBP_ring.drop(['TRG'],axis = 1), rsuffix='_hbp_ring')\
                   .join(PVP_core.drop(['TRG'],axis = 1), rsuffix='_pvp_core')\
                   .join(PVP_ring.drop(['TRG'],axis = 1), rsuffix='_pvp_pring')


In [5]:
from sklearn.base import BaseEstimator,TransformerMixin

class MyDecorrelator(BaseEstimator, TransformerMixin):
    
    def __init__(self, threshold):
        self.threshold = threshold
        self.correlated_columns = None

    def fit(self, X, y=None):
        correlated_features = set()  
        X = pd.DataFrame(X)
        corr_matrix = X.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > self.threshold: # we are interested in absolute coeff value
                    colname = corr_matrix.columns[i]  # getting the name of column
                    correlated_features.add(colname)
        self.correlated_features = correlated_features
        return self

    def transform(self, X, y=None, **kwargs):
        return (pd.DataFrame(X)).drop(labels=self.correlated_features, axis=1)

In [6]:
dataset = epato_core.copy()

# train-test-validation esterna
from sklearn.model_selection import RepeatedKFold
def dataset_split(dataset):

    dt = dataset.drop(['TRG'],axis = 1)
    X_interno, X_esterno, y_interno, y_esterno = train_test_split(dt, dataset['TRG'], test_size=0.33, random_state=42, stratify= dataset['TRG'])
    return X_interno,y_interno,X_esterno,y_esterno

# Optimization module

def objective(trial,X,y):

    
    # Standardization
    

    
    std = trial.suggest_categorical('standard', ['StandardScaler', 'MinMaxScaler', 'PowerTransformer'])
    
    if std=='StandardScaler':
        scaler = StandardScaler()
    elif std=='MinMaxScaler':
        scaler = MinMaxScaler()
    else:
        scaler = PowerTransformer()
    

    dec_perc = trial.suggest_float('dec_perc',0.8,0.99, step = 0.02)
    dec = MyDecorrelator(dec_perc)

    
    # Feature selection
    
    fs = trial.suggest_categorical('feature_selection', [  'SelectPercentile'])
    
    if fs=='VarianceThreshold':
        threshold_fs = trial.suggest_float('threshold_fs',0,1, step = 0.05)
        selector = VarianceThreshold(threshold_fs)

    elif fs=='SequentialFeatureSelector':
        estim = LogisticRegression(n_jobs=-1)
        n_features_to_select_fs = trial.suggest_int('n_features_to_select_fs',4,20)
        direction_fs = trial.suggest_categorical('direction_fs', ['forward'])
        selector = SequentialFeatureSelector(estim,
                                             n_features_to_select = n_features_to_select_fs,
                                             direction = direction_fs,scoring = 'roc_auc',n_jobs = -1)

    else:
        percentile_fs = trial.suggest_float('percentile_fs',5,15, step = 1)
        selector = SelectPercentile(mutual_info_classif,percentile = percentile_fs)

    
    
    # Model
    m = trial.suggest_categorical('model', ['SGDClassifier','LinearSVC','AdaBoostClassifier','RandomForestClassifier','GradientBoostingClassifier'])
    
    if m == 'SGDClassifier':
        loss_sgd = trial.suggest_float('loss_sgd',1e-2,1e2, step = 0.001)
        m = LogisticRegression(C=loss_sgd,solver = 'liblinear',class_weight='balanced')
    elif m == 'LinearSVC':
        m = LinearSVC(class_weight='balanced')
    elif m == 'AdaBoostClassifier':
        criterion_ada = trial.suggest_categorical('criterion_ada', ['gini', 'entropy'])
        max_depth_ada = trial.suggest_int('max_depth_ada',2,5)
        base_estimator = DecisionTreeClassifier(criterion = criterion_ada,
                                                max_depth=max_depth_ada,class_weight='balanced')
        n_estimators_ada = trial.suggest_int('n_estimators_ada',3,10)
        learning_rate_ada = trial.suggest_float('learning_rate_ada',0.1,10)
        m = AdaBoostClassifier(estimator = base_estimator, n_estimators=n_estimators_ada,
                               learning_rate=learning_rate_ada,random_state=4)
    elif m == 'RandomForestClassifier':
        n_estimators_rf =trial.suggest_int('n_estimators_rf',300,500)
        criterion=trial.suggest_categorical('criterion_rf', ['gini', 'entropy'])
        max_depth=trial.suggest_int('max_depth_rf',2,5)
        min_samples_split=trial.suggest_int('min_samples_split_rf',2,10)
        min_samples_leaf=trial.suggest_int('min_samples_leaf_rf',2,10)
        max_features=trial.suggest_float('max_features_rf',0.3,0.6)
        #ccp_alpha=trial.suggest_float('ccp_alpha_rf',1e-5,1)
        max_samples=trial.suggest_float('max_samples_rf',0.5,0.9)
        #print(n_estimators_rf)
        m = RandomForestClassifier(n_estimators=n_estimators_rf,
                                   criterion=criterion,
                                   max_depth=max_depth,
                                   #min_samples_leaf=min_samples_leaf,
                                   #min_samples_split = min_samples_split,
                                   max_features=max_features,
                                   #max_samples=max_samples,
                                   #class_weight='balanced_subsample',
                                   n_jobs=-1)
    else:
        m = GradientBoostingClassifier()
        
    cv = StratifiedKFold(5)
    cvc = CalibratedClassifierCV(m, cv=cv, method="sigmoid")
    pipe = Pipeline([('scaler',scaler),('decor',dec),('selector',selector),('model',m)])
    
    accuracy = cross_val_score(pipe,X,y, scoring='roc_auc',cv = RepeatedKFold(n_splits=3, n_repeats=2,), n_jobs=-1,error_score='raise')
    
    
    return np.mean(accuracy)

# 3. Create a study object and optimize the objective function.
def train(X,y):
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    func = lambda trial: objective(trial, X, y)
    study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler())
    study.optimize(func, n_trials=100)
    return study

In [7]:
def evaluate(best_params,X_interno,y_interno,X_esterno,y_esterno):

    std = best_params['standard']
    if std=='StandardScaler':
            scaler = StandardScaler()
    elif std=='MinMaxScaler':
            scaler = MinMaxScaler()
    else:
            scaler = PowerTransformer()


    dec = MyDecorrelator(best_params['dec_perc'])
        # Feature selection

    fs = best_params['feature_selection']

    if fs=='VarianceThreshold':
            threshold_fs = best_params['threshold_fs']
            selector = VarianceThreshold(threshold_fs)
    elif fs=='SequentialFeatureSelector':
            estim = LogisticRegression(n_jobs=-1)
            n_features_to_select_fs = best_params['n_features_to_select_fs']
            direction_fs = best_params['direction_fs']
            selector = SequentialFeatureSelector(estim,
                                                 n_features_to_select = n_features_to_select_fs,
                                                 direction = direction_fs,scoring = 'roc_auc',n_jobs = -1)

    else:
            percentile_fs = best_params['percentile_fs']
            selector = SelectPercentile(mutual_info_classif,percentile = percentile_fs)


        # Model
    m = best_params['model']

    if m == 'SGDClassifier':
            loss_sgd = best_params['loss_sgd']
            m = LogisticRegression(C=loss_sgd,solver = 'liblinear',class_weight='balanced',n_jobs=-1)
    elif m == 'LinearSVC':
            m = LinearSVC(class_weight='balanced')
    elif m == 'AdaBoostClassifier':
            base_estimator = DecisionTreeClassifier(criterion = best_params['criterion_ada'],
                                                    max_depth=best_params['max_depth_ada'])

            m = AdaBoostClassifier(estimator = base_estimator, n_estimators=best_params['n_estimators_ada'],
                                   learning_rate=best_params['learning_rate_ada'],random_state=4)
            #m = AdaBoostClassifier()
    elif m == 'RandomForestClassifier':
            
            m = RandomForestClassifier(n_estimators=best_params['n_estimators_rf'],
                                   criterion=best_params['criterion_rf'],
                                    max_depth=best_params['max_depth_rf'],
                                   #min_samples_leaf=best_params['min_samples_leaf_rf'],
                                   #min_samples_split = best_params['min_samples_split_rf'],
                                   max_features=best_params['max_features_rf'],
                                   #max_samples=best_params['max_samples_rf'],
                                   #ccp_alpha=best_params['ccp_alpha_rf'],
                                   #class_weight='balanced_subsample',
                                       n_jobs=-1)
    else:
            m = GradientBoostingClassifier()

    cv = StratifiedKFold(5)
    cvc = CalibratedClassifierCV(m, cv=cv, method="sigmoid")
    
    pipe = Pipeline([('scaler',scaler),('decor',dec),('selector',selector),('model',m)])
    
    
    
    
    cvc = pipe.fit(X_interno,y_interno)
    print('Train\n')

    y_pred_train = cvc.predict(X_interno)
    y_pred_proba_train = cvc.predict_proba(X_interno)

    
    acc = accuracy_score(y_interno,y_pred_train)
    sens = recall_score(y_interno,y_pred_train)
    spec = recall_score(y_interno,y_pred_train,pos_label=0)
    auc = roc_auc_score(y_interno,y_pred_proba_train[:,1])

    
    #scores = cross_val_score(pipe,X, y, cv=10,scoring='accuracy')
    #print("Accuracy: %0.2f (%0.2f-%0.2f)" % (scores.mean(), np.quantile(scores,q= 0.25),np.quantile(scores,q= 0.75)))

    
    print('Acc = ',round(acc,3),' Sens = ',round(sens,3), ' Spec = ',round(spec,3),' AUC = ',round(auc,3))
    print('\n\n')
    print('Test\n')

    y_pred_test = cvc.predict(X_esterno)
    y_pred_proba_test = cvc.predict_proba(X_esterno)
    acc = accuracy_score(y_esterno,y_pred_test)
    sens = recall_score(y_esterno,y_pred_test)
    spec = recall_score(y_esterno,y_pred_test,pos_label=0)
    auc = roc_auc_score(y_esterno,y_pred_proba_test[:,1])
    print('Acc = ',round(acc,3),' Sens = ',round(sens,3), ' Spec = ',round(spec,3),' AUC = ',round(auc,3))
    return cvc,m

In [10]:
list_datasets = [epato_core,epato_ring,portale_core,portale_ring, core, ring, epato, portale, totale]
dict_results = {}

for i, dataset in enumerate(list_datasets):
    if i == 0:
        t = 'HBP_core'
        
    elif i == 1:
        t = 'HBP_ring'
        
    elif i == 2:
        t = 'PVP_core'
        
    elif i == 3:
        t = 'PVP_ring'
        
    elif i == 4:
        t = 'Core total'
        
    elif i == 5:
        t = 'Ring total'
        
    elif i == 6:
        t = 'HBP totale'
        
    elif i == 7:
        t = 'Po totale'
        
    else:
        t = 'Totale'
        
    if t=='Totale':
        
        print('-- Dataset Analysis: ',t)
        X_interno,y_interno,X_esterno,y_esterno = dataset_split(dataset)
        study = train(X_interno,y_interno)
        display(study.best_params)
        cvc, m = evaluate(study.best_params,X_interno,y_interno,X_esterno,y_esterno)
        s = pickle.dump(cvc,open(t+'_CalibratedClassifier_Radiomics.pkl', 'wb'))
        s = pickle.dump(cvc,open(t+'_Classifier_Radiomics.pkl', 'wb'))
        

        print('\n ')

-- Dataset Analysis:  Totale


/home/riccardo/miniconda3/lib/python3.9/site-packages/optuna/distributions.py:685: UserWarning: The distribution is specified by [0.8, 0.99] and step=0.02, but the range is not divisible by `step`. It will be replaced by [0.8, 0.98].
  warnings.warn(


{'standard': 'MinMaxScaler',
 'dec_perc': 0.88,
 'feature_selection': 'SelectPercentile',
 'percentile_fs': 10.0,
 'model': 'SGDClassifier',
 'loss_sgd': 2.1039999999999996}

Train

Acc =  0.752  Sens =  0.717  Spec =  0.782  AUC =  0.806



Test

Acc =  0.686  Sens =  0.652  Spec =  0.714  AUC =  0.691

 


/home/riccardo/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1223: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 24.
  warnings.warn(
